<a href="https://colab.research.google.com/github/vipul43/image_description/blob/main/MutliLabelClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import scipy.io
import cv2
from google.colab.patches import cv2_imshow
from skimage import io

import numpy as np
import pandas as pd
import cv2 as cv 
from google.colab.patches import cv2_imshow # for image display
from skimage import io
from PIL import Image 
import matplotlib.pylab as plt

In [ ]:
mat = scipy.io.loadmat('/content/drive/MyDrive/Project Dataset/abstract50S.mat')
type(mat)

In [ ]:
j = 0
for i in mat["abs_sent"][0]:
  print(i)
  break

In [ ]:
mat02 = scipy.io.loadmat('/content/drive/MyDrive/Project Dataset/pascal50S.mat')


In [ ]:

classes = ['person', 
           'bird', 
           'cat', 
           'cow', 
           'dog', 
           'horse', 
           'sheep', 
           'aeroplane', 
           'bicycle', 
           'boat', 
           'bus', 
           'car', 
           'motorbike', 
           'train', 
           'bottle', 
           'chair', 
           'dining table', 
           'potted plant', 
           'sofa', 
           'tv/monitor']



In [ ]:
j = 0

for i in mat02["train_sent_final"][0]:
  # image = io.imread(i[0][0])
  # cv2_imshow(image)
  for k in i[1]:
    for sent in k:
      print(sent[0])
      for word in sent[0].split():
        if(word in classes):
          print(word)
        
  if j > 1:
    break
  j = j + 1

In [ ]:
j = 0

for i in mat["abs_sent"][0]:
  # image = io.imread(i[0][0])
  # cv2_imshow(image)
  for k in i[1]:
    for sent in k:
      print(sent[0])
      for word in sent[0].split():
        if(word in classes):
          print(word)
        
  if j > 1:
    break
  j = j + 1

h
t
t
p
:
/
/
f
i
l
e
b
o
x
.
e
c
e
.
v
t
.
e
d
u
/
~
v
r
a
m
a
9
1
/
c
l
i
p
a
r
t
/
S
c
e
n
e
8
9
8
_
0
.
p
n
g
h
t
t
p
:
/
/
f
i
l
e
b
o
x
.
e
c
e
.
v
t
.
e
d
u
/
~
v
r
a
m
a
9
1
/
c
l
i
p
a
r
t
/
S
c
e
n
e
4
7
6
_
0
.
p
n
g
h
t
t
p
:
/
/
f
i
l
e
b
o
x
.
e
c
e
.
v
t
.
e
d
u
/
~
v
r
a
m
a
9
1
/
c
l
i
p
a
r
t
/
S
c
e
n
e
6
3
3
_
0
.
p
n
g


In [ ]:
len(mat02["train_sent_final"])

1

In [ ]:
len(mat02["train_sent_final"][0])

1000

In [ ]:
len(mat02["train_sent_final"][0][0])

2

In [ ]:
len(mat02["train_sent_final"][0][0][0])

1

In [ ]:
len(mat02["train_sent_final"][0][0][1])

1

In [ ]:
len(mat02["train_sent_final"][0][0][0][0])

58

In [ ]:
len(mat02["train_sent_final"][0][0][0][0][0])

1

In [ ]:
len(mat02["train_sent_final"][0][0][0][0][0][0])

1

In [ ]:
len(mat02["train_sent_final"][0][0][0][0][0][0][0])

1